# Finding the volume of a Pareto frontier
Given a Pareto front, estimates the volume "underneath" the frontier (towards the ideal solution) through simulation

### Pre-processing

In [205]:
import pandas as pd
import numpy as np

In [251]:
# Get the solutions that define the pareto frontier.
# It is assumed that these frontiers do not contain any dominated solutions
# (that is, that the solutions are truly Pareto optimal)

sols = pd.read_csv("solutionSets/3d/climateChange_EfficientSolutions_NoneOnly.csv")

In [252]:
# Hard-coded manipulations to tidy-up sols

sols = sols.drop(["SolutionIndex","Frontier"],axis=1) #remove solution index and frontier column

In [253]:
# Get list of objectives from the dataframe

objs = sols.columns.tolist()

In [254]:
# Create empty dictionary to hold the objectives and the senses of each
# This process is hard-coded and requires some knowledge about the model
# that resulted in the sols dataframe

objSenses = {}

In [255]:
# Hard-coded manipulations to properly set objSenses (max/min)
# 1 for max, and 0 for min

for obj in objs:
    objSenses[obj] = 0 # all objectives are min for curr example
objSenses["Northern Spotted Owl Habitat (ha)"] = 1

In [256]:
# Get the objectives' bounds

objBounds = {}
for obj,sense in objSenses.items():
    objBounds[obj] = [sols[obj].min(),sols[obj].max()]

In [257]:
# Get each objective's ideal value

idealObjVals = {}
for obj,bounds in objBounds.items():
    idealObjVals[obj] = bounds[objSenses[obj]] # 0th entry of bounds is min, 1st is max

In [258]:
# Get each objective's worst value

worstObjVals = {}
for obj,bounds in objBounds.items():
    worstObjVals[obj] = bounds[not(objSenses[obj])]

In [259]:
# Compute the non-normalized volume of the objective space

objSpaceVolume_nonNorm = 1
for obj,bounds in objBounds.items():
    objSpaceVolume_nonNorm *= bounds[1] - bounds[0]

In [260]:
# Normalize the objective space by converting each value to the relative achievement along its axis:

#   distance from the worst case value
# ---------------------------------------
# total distance spanned by the objective

# Objective space is now in [0,1]^N (N is number of objectives)


for obj,bounds in objBounds.items():
    sols[obj] = sols[obj].apply(
        lambda x: \
        abs(x - worstObjVals[obj]) / \
        (bounds[1]-bounds[0]))

### Define hyperrectangle class

In [261]:
class HyperRectangle_Origin:
    """ A class to manufacture hyperrectangle (N-dimensional rectangle) objects
    in which all dimensions live in positive real numbers (R^+)"""

    def __init__(self, farCorner):
        """ Initialize hyperrectangle with extreme points at the origin and farCorner"""
        self.n = len(farCorner)
        self.origin = np.zeros(self.n)
        self.farCorner = farCorner
        self.volume = np.prod(farCorner)
    
    def __str__(self):
        result = "(" + str(self.farCorner[0])
        for coord in self.farCorner[1:]:
            result += ", " + str(coord)
        return  result + ")" 
        

    def contains(self, point):
        """ Determine whether point lies within the hyperrectangle """
        if (np.array(point) < 0).any():
            return False
        else:
            return (np.array(point) <= self.farCorner).all()

### Begin simulation to estimate volume

In [262]:
# Number of simulations to perform
# We repeat the simulation process to determine an adequate burn-in period
 
N = np.arange(0,30000,1000)

In [263]:
# Create dataframe to hold #simulations:volume relationship

volsim = pd.DataFrame(index=N, columns=["Volume"])

In [264]:
# Instantiate series of hyperrectangles

hrects = []

In [265]:
# To the frontier, add a hyperrectangle for each solution

for idx, row in sols.iterrows():
    hrects.append(HyperRectangle_Origin(row[objs].tolist()))

In [266]:
# Store the number of dimensions

dims = hrects[0].n

In [267]:
# Estimate the volume by counting the number of points that fall the series of hyperrectangles

for idx,row in volsim.iterrows():
    numSim = idx
    numWithin = 0
    for n in range(numSim):
        # generate random point and see if it falls within the frontier
        rand = np.random.rand(dims)
        numWithin += len([hrect for hrect in hrects if hrect.contains(rand)])>0
    volsim.set_value(idx,"Volume",numWithin/numSim)

In [269]:
#volsim.to_csv("solutionSets/3d/ClimateChange_None_VolumeSimming.csv")